Before you begin, execute this cell to import numpy and packages from the D-Wave Ocean suite, and all necessary functions the gate-model framework you are going to use, whether that is the Forest SDK or Qiskit. In the case of Forest SDK, it also starts the qvm and quilc servers.

In [130]:
%run -i "assignment_helper.py"

Available frameworks:
Forest SDK
Qiskit
D-Wave Ocean


# State preparation

Preparing a state in a particular encoding can, in itself, give rise to interesting kernels. This is what talked about in the lecture on the interference kernel, and Maria Schuld's guest lecture expands on the idea. Let us work with an easy data set of two vectors, $S = \{(\begin{bmatrix}0 \\ 1\end{bmatrix}, 0), (\begin{bmatrix}\sqrt{2}/2 \\ \sqrt{2}/2\end{bmatrix}, 1)\}$. Let's have a test instance $\begin{bmatrix}1 \\ 0\end{bmatrix}$. We will build the interference circuit for this.

**Exercise 1** (1 point). Create a circuit in your preferred framework that works on four qubits: ancilla, index, data, and class. Put the ancilla and index qubits into a uniform superposition. Prepare and entangle the test instance with the ground state of the ancilla. Put an identity gate on the class qubit. Place your solution in an object called `circuit`. 

In [131]:
from math import pi

q = QuantumRegister(4)
c = ClassicalRegister(4)

ancilla_qubit = q[0]
index_qubit = q[1]
data_qubit = q[2]
class_qubit = q[3]

###
circuit = QuantumCircuit(q, c)

# Put the ancilla and the index qubits into uniform superposition
circuit.h(ancilla_qubit)
circuit.h(index_qubit)

# Prepare the test vector
circuit.cx(ancilla_qubit, data_qubit)
circuit.x(data_qubit)
#circuit.cx(ancilla_qubit, data_qubit)
#circuit.u3(pi/2,pi/2,pi/2, data_qubit)
#circuit.x(ancilla_qubit)

#circuit.iden(class_qubit)
circuit.barrier()

amplitudes = get_amplitudes(circuit)
print(amplitudes)

circuit.draw()

###

[0. +0.j 0.5+0.j 0. +0.j 0.5+0.j 0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0. +0.j
 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]


In [132]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([0. +0.j, 0.5+0.j, 0. +0.j, 0.5+0.j, 0.5+0.j, 0. +0.j, 0.5+0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j]))

**Exercise 2** (1 point). Extend the circuit to prepare the first training instance and entanle it with the excited state of the ancilla and ground state of the index qubit.

In [133]:
###
# Prepare the first training vector
# [0,1] -> class 0
# We can prepare this with a Toffoli
circuit.ccx(ancilla_qubit, index_qubit, data_qubit)

# Flip the index qubit > moves the first training vector to the 
    # |0> state of the index qubit
circuit.x(index_qubit)
circuit.barrier()

amplitudes = get_amplitudes(circuit)
print(amplitudes)

circuit.draw()
###


[0. +0.j 0. +0.j 0. +0.j 0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j 0. +0.j 0. +0.j
 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]


In [134]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([0. +0.j, 0. +0.j, 0. +0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j]))

**Exercise 3** (2 points). Extend the circuit to prepare the second training instance and entangle it with the excited state of the ancilla and the excited of the index qubit.

In [135]:
###
circuit.ccx(ancilla_qubit, index_qubit, data_qubit)
circuit.cx(index_qubit, data_qubit)
circuit.ry(pi/2, data_qubit)
circuit.cx(index_qubit, data_qubit)
circuit.ry(-pi/2, data_qubit)
circuit.ccx(ancilla_qubit, index_qubit, data_qubit)
circuit.cx(index_qubit, data_qubit)
circuit.ry(pi/2, data_qubit)
circuit.cx(index_qubit, data_qubit)
circuit.ry(-pi/2, data_qubit)
circuit.barrier()


amplitudes = get_amplitudes(circuit)
print(amplitudes)

circuit.draw()
###


[ 0. +0.j  0. +0.j  0. +0.j  0.5+0.j  0.5+0.j  0.5+0.j -0.5+0.j  0. +0.j
  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j]


In [136]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([ 0. +0.j,  0. +0.j,  0. +0.j,  0.5+0.j,  0.5+0.j,  0.5+0.j,
       -0.5+0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j, 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j]))

**Exercise 4** (1 point). Finish the state preparation circuit by flipping the class qubit conditioned on the index qubit.

In [137]:
###
# Flip the class label for training vector #2
circuit.cx(index_qubit, class_qubit)
circuit.barrier()

amplitudes = get_amplitudes(circuit)
print(amplitudes)

circuit.draw()
###


[ 0. +0.j  0. +0.j  0. +0.j  0. +0.j  0.5+0.j  0.5+0.j  0. +0.j  0. +0.j
  0. +0.j  0. +0.j  0. +0.j  0.5+0.j  0. +0.j  0. +0.j -0.5+0.j  0. +0.j]


In [138]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0.5+0.j,  0.5+0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0.5+0.j, 0. +0.j,  0. +0.j, -0.5+0.j,  0. +0.j]))

# Interference as a kernel

**Exercise 5** (1 point). At this point, our state is $\frac{1}{2\sqrt{2}}\sum_{i=0}^1|0\rangle|i\rangle(|x_t\rangle+|x_i\rangle)|y_i\rangle+|1\rangle|i\rangle(|x_t\rangle-|x_i\rangle)|y_i\rangle$, where $|x_t\rangle$ is the encoded test instance and $|x_i\rangle$ is a training instance. Apply the Hadamard gate on the ancilla to apply the interference.

In [139]:
###
circuit.h(q[0])
circuit.barrier()
###


In [140]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([ 0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
        0.70710678+0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,  0.35355339+0.j, -0.35355339+0.j,
        0.        +0.j,  0.        +0.j, -0.35355339+0.j, -0.35355339+0.j]))

If we measure the ancilla, the outcome probability of observing 0 will be $\frac{1}{4N}\sum_{i=1}^N |x_t + x_i|^2$. Performing post-selection on the 0 outcome, we can calculate the kernel and the probability of the test instance belonging to either class.